<a href="https://colab.research.google.com/github/Polapob/Quora-text-classification/blob/main/%E0%B8%AA%E0%B8%B3%E0%B9%80%E0%B8%99%E0%B8%B2%E0%B8%82%E0%B8%AD%E0%B8%87_Quora_summary_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import copy
import time
import pandas as pd
import numpy as np
import gc
import re
import tensorflow as tf
from torchtext import data
#import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')
from collections import Counter
from textblob import TextBlob
from nltk import word_tokenize


from sklearn.metrics import f1_score
import torchtext
import os 

# cross validation and metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from textblob import TextBlob
from multiprocessing import  Pool
from functools import partial
import numpy as np



In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c quora-insincere-questions-classification

 70% 11.0M/15.8M [00:00<00:00, 36.0MB/s]
100% 15.8M/15.8M [00:00<00:00, 45.4MB/s]
 89% 49.0M/54.9M [00:01<00:00, 47.5MB/s]
100% 54.9M/54.9M [00:01<00:00, 47.3MB/s]
  0% 0.00/4.09M [00:00<?, ?B/s]
100% 4.09M/4.09M [00:00<00:00, 37.3MB/s]
100% 5.96G/5.96G [02:12<00:00, 27.0MB/s]
100% 5.96G/5.96G [02:12<00:00, 48.4MB/s]


In [ ]:
! mkdir train
! mkdir test1
! mkdir submission
! mkdir embedding

In [ ]:
! unzip train.csv.zip -d train
! unzip test.csv.zip -d test
! unzip sample_submission.csv.zip -d submission
! unzip embeddings.zip -d embedding

Archive:  train.csv.zip
  inflating: train/train.csv         
Archive:  test.csv.zip
  inflating: test/test.csv           
Archive:  sample_submission.csv.zip
  inflating: submission/sample_submission.csv  
Archive:  embeddings.zip
   creating: embedding/GoogleNews-vectors-negative300/
   creating: embedding/glove.840B.300d/
   creating: embedding/paragram_300_sl999/
   creating: embedding/wiki-news-300d-1M/
  inflating: embedding/glove.840B.300d/glove.840B.300d.txt  

In [ ]:
! unzip embeddings.zip -d embedding

Archive:  embeddings.zip
replace embedding/glove.840B.300d/glove.840B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
  inflating: embedding/paragram_300_sl999/README.txt  
  inflating: embedding/paragram_300_sl999/paragram_300_sl999.txt  

In [ ]:
! unzip embeddings.zip -d embedding 

Archive:  embeddings.zip
replace embedding/glove.840B.300d/glove.840B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
  inflating: embedding/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin  

In [ ]:
! unzip embeddings.zip -d embedding

Archive:  embeddings.zip
replace embedding/glove.840B.300d/glove.840B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
  inflating: embedding/wiki-news-300d-1M/wiki-news-300d-1M.vec  

In [ ]:
train = pd.read_csv("train/train.csv")
test = pd.read_csv("test/test.csv")
submission = pd.read_csv("submission/sample_submission.csv")

In [ ]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';',  '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x

def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
        x = re.sub('[0-9]{1}', '#', x)
    return x
clean_train = train.copy()
clean_train["question_text"]= clean_train.question_text.apply(lambda x: clean_text(x))
clean_train["question_text"]= clean_train.question_text.apply(lambda x: clean_numbers(x))

In [ ]:
missspell_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                "'cause": "because", "could've": "could have", "couldn't": "could not",
                "didn't": "did not",  "doesn't": "does not", "don't": "do not", 
                "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
                "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", 
                "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am",
                "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", 
                "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
                "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is",
                "that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", 
                "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are",
                "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  
                "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", 
                "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are",
                "y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will",
                "you'll've": "you will have", "you're": "you are", "you've": "you have", 'colour': 'color', 'centre': 'center',
                'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2',
                'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What',
                'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much',
                'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis',
                'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota',
                'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization',
                'demonitization': 'demonetization', 'demonetisation': 'demonetization'}

In [ ]:
def sort_dictionary(list_sentence,lower):
  dictionary1 = {}
  if lower == False:
    for sentence in list_sentence:
      for word in sentence.split():
        try:
          dictionary1[word] += 1
        except:
          dictionary1[word] = 1
    word = [[word,occurrence] for word,occurrence in dictionary1.items()]
    word = sorted(word,key = lambda x:x[1],reverse=True)
    sort_dict = {}
    for i in word:
      sort_dict[i[0]] = i[1]
  elif lower == True:
    for sentence in list_sentence:
      for word in sentence.split():
        str1 = str(word).lower()
        try:
          dictionary1[str1] += 1
        except:
          dictionary1[str1] = 1
    word = [[word,occurrence] for word,occurrence in dictionary1.items()]
    word = sorted(word,key = lambda x:x[1],reverse=True)
    sort_dict = {}
    for i in word:
      sort_dict[i[0]] = i[1]
  return sort_dict

In [ ]:
list2 = []

for i in range(2):
  dict1 = sort_dictionary(clean_train["question_text"].values,lower=False)
  print("Before improvement")
  print("Len missspell dict = " + str(len(missspell_dict)))
  for i in missspell_dict:
    try:
      str1 = i[0].upper() +i[1:]
      if str1 in dict1:
        #print(str1 + " doesn't full clean.")
        r = True
      list2.append(str1.lower())
    except:
      pass
  lista = list2[:len(list2)-1]
  for i in lista:
    try:
      str1 = i[0].upper() +i[1:]    
      str2 = missspell_dict[i][0].upper() + missspell_dict[i][1:]
      missspell_dict[str1] = str2 
    except:
      pass
  print("After improvement")
  print("Len missspell dict = " + str(len(missspell_dict)))
  print()

Before improvement
Len missspell dict = 162
After improvement
Len missspell dict = 303

Before improvement
Len missspell dict = 303
After improvement
Len missspell dict = 304



In [ ]:
list2 = []
for i in missspell_dict:
  try:
    str1 = i[0].upper() +i[1:]
    if str1 in dict1:
      #print(str1 + " doesn't full clean.")
      r = True
    list2.append(str1.lower())
  except:
      pass
for i in lista:
    try:
      str1 = i[0].upper() +i[1:]    
      str2 = missspell_dict[i][0].upper() + missspell_dict[i][1:]
      missspell_dict[str1] = str2 
    except:
      print(str1,str2)
      pass

Qoura Youtube 
Whta Salary
Doi Why do
Thebest Why do
Etherium Penis
Qoura Youtube 
Whta Salary
Doi Why do
Thebest Why do
Etherium Penis


In [ ]:
def check_missspell(text):
  str_text = str(text)
  for i in missspell_dict:
    if i in str_text:
      str_text = str_text.replace(i,missspell_dict[i])
  return str_text

In [ ]:
clean_train["question_text"] = clean_train["question_text"].apply(lambda x: check_missspell(x))

In [ ]:
Start_X = clean_train.question_text.values
Start_Y = clean_train.target.values

In [ ]:
list_sentence = Start_X

In [ ]:
vocab_dictionary = {}
for sentence in list_sentence:
  for word in sentence:
    try:
      vocab_dictionary[word]+=1
    except:
      vocab_dictionary[word]=1

In [ ]:
X = []
for i in Start_X:
  X.append(str(i))
X[:10]

['How did Quebec nationalists see their province as a nation in the ####s ? ',
 'Do you have an adopted dog ,  how would you encourage people to adopt and not shop ? ',
 'Why does velocity affect time ?  Does velocity affect space geometry ? ',
 'How did Otto von Guericke used the Magdeburg hemispheres ? ',
 'Can I convert montra helicon D to a mountain bike by just changing the tyres ? ',
 'Is Gaza slowly becoming Auschwitz ,  Dachau or Treblinka for Palestinians ? ',
 'Why does Quora automatically ban conservative opinions when reported ,  but does not do the same for liberal views ? ',
 'Is it crazy if I wash or wipe my groceries off ?  Germs are everywhere . ',
 'Is there such a thing as dressing moderately ,  and if so ,  how is that different than dressing modestly ? ',
 'Is it just me or have you ever been in this phase wherein you became ignorant to the people you once loved ,  completely disregarding their feelings / lives so you get to have something go your way and feel temp

In [ ]:
Start_X

array(['How did Quebec nationalists see their province as a nation in the ####s ? ',
       'Do you have an adopted dog ,  how would you encourage people to adopt and not shop ? ',
       'Why does velocity affect time ?  Does velocity affect space geometry ? ',
       ..., 'Is foam insulation toxic ? ',
       'How can one start a research project based on biochemistry at UG level ? ',
       'Who wins in a battle between a Wolverine and a Puma ? '],
      dtype=object)

In [ ]:
print("Check the number of vocab with lower occurrence \n")
occurrence = 31
dict1 = sort_dictionary(X,lower=False)
for i in range(1,occurrence+1):
  dict_check = {}
  count = 1
  for word in dict1:
    if dict1[word] > i-1:
      dict_check[word] = count
      count += 1
  print("Check word of dictionary that occur more than " + str(i-1)+" times." +"\n")
  print("The number of word is " + str(len(dict_check))+"\n")

Check the number of vocab with lower occurrence 

Check word of dictionary that occur more than 0 times.

The number of word is 246534

Check word of dictionary that occur more than 1 times.

The number of word is 110987

Check word of dictionary that occur more than 2 times.

The number of word is 80043

Check word of dictionary that occur more than 3 times.

The number of word is 65728

Check word of dictionary that occur more than 4 times.

The number of word is 56977

Check word of dictionary that occur more than 5 times.

The number of word is 51003

Check word of dictionary that occur more than 6 times.

The number of word is 46415

Check word of dictionary that occur more than 7 times.

The number of word is 42943

Check word of dictionary that occur more than 8 times.

The number of word is 40178

Check word of dictionary that occur more than 9 times.

The number of word is 37824

Check word of dictionary that occur more than 10 times.

The number of word is 35827

Check word o

First We use based Model to predict outcome. So we want case unsensitive word

In [ ]:
list_sentence = []
for i in Start_X:
  list_word = []
  for j in i.split():
    list_word.append(j.lower())
  list_sentence.append(list_word)

In [ ]:
list_sentence[:20]

[['how',
  'did',
  'quebec',
  'nationalists',
  'see',
  'their',
  'province',
  'as',
  'a',
  'nation',
  'in',
  'the',
  '####s',
  '?'],
 ['do',
  'you',
  'have',
  'an',
  'adopted',
  'dog',
  ',',
  'how',
  'would',
  'you',
  'encourage',
  'people',
  'to',
  'adopt',
  'and',
  'not',
  'shop',
  '?'],
 ['why',
  'does',
  'velocity',
  'affect',
  'time',
  '?',
  'does',
  'velocity',
  'affect',
  'space',
  'geometry',
  '?'],
 ['how',
  'did',
  'otto',
  'von',
  'guericke',
  'used',
  'the',
  'magdeburg',
  'hemispheres',
  '?'],
 ['can',
  'i',
  'convert',
  'montra',
  'helicon',
  'd',
  'to',
  'a',
  'mountain',
  'bike',
  'by',
  'just',
  'changing',
  'the',
  'tyres',
  '?'],
 ['is',
  'gaza',
  'slowly',
  'becoming',
  'auschwitz',
  ',',
  'dachau',
  'or',
  'treblinka',
  'for',
  'palestinians',
  '?'],
 ['why',
  'does',
  'quora',
  'automatically',
  'ban',
  'conservative',
  'opinions',
  'when',
  'reported',
  ',',
  'but',
  'does',
  '

In [ ]:
dict2["what's"]

In [ ]:
print("Check the number of vocab with lower occurrence \n")
occurrence = 31
dict2 = sort_dictionary(X,lower=True)
for i in range(1,occurrence+1):
  dict_check = {}
  count = 1
  for word in dict2:
    if dict2[word] > i-1:
      dict_check[word] = count
      count += 1
  print("Check word of dictionary that occur more than " + str(i-1)+" times." +"\n")
  print("The number of word is " + str(len(dict_check))+"\n")

Check the number of vocab with lower occurrence 

Check word of dictionary that occur more than 0 times.

The number of word is 202942

Check word of dictionary that occur more than 1 times.

The number of word is 94456

Check word of dictionary that occur more than 2 times.

The number of word is 69278

Check word of dictionary that occur more than 3 times.

The number of word is 57695

Check word of dictionary that occur more than 4 times.

The number of word is 50488

Check word of dictionary that occur more than 5 times.

The number of word is 45538

Check word of dictionary that occur more than 6 times.

The number of word is 41700

Check word of dictionary that occur more than 7 times.

The number of word is 38739

Check word of dictionary that occur more than 8 times.

The number of word is 36288

Check word of dictionary that occur more than 9 times.

The number of word is 34247

Check word of dictionary that occur more than 10 times.

The number of word is 32519

Check word of

In [ ]:
dict1 = sort_dictionary(X,lower=False)
dict2 = sort_dictionary(X,lower=True)

In [ ]:
number_of_word = 100000
dict_use = {"OOV":0}
count = 1
for i in dict1:
  count +=1
  dict_use[i] =len(dict_use)
  if count ==100000:
    break

In [ ]:
max_word_per_sentence = 70
padding = []
for sentence in X:
  list2 =[]
  for word in sentence.split():
    try:
      list2.append(dict_use[word])
    except:
      list2.append(dict_use["OOV"])
  padding.append(list2)

In [ ]:
max_word_per_sentence = 70
padding_maxword = []
for i in padding:
  sentence  = i.copy()
  if len(i) > max_word_per_sentence:
    padding_maxword.append(sentence[:max_word_per_sentence])
  else:
    for i in range(max_word_per_sentence-len(i)):
      sentence.append(0)
    padding_maxword.append(sentence)

In [ ]:
for i in padding_maxword[:10]:
  print(i)

[10, 66, 6949, 8033, 183, 68, 6664, 42, 5, 1266, 6, 2, 1564, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[58, 16, 26, 36, 3871, 522, 12, 79, 46, 16, 3640, 44, 4, 3067, 11, 29, 1899, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[17, 32, 2023, 390, 89, 1, 90, 2023, 390, 515, 5593, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[10, 66, 14194, 10404, 56978, 139, 2, 42944, 30063, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[48, 9, 1119, 65729, 0, 1227, 4, 5, 4045, 1588, 56, 120, 1485, 2, 1

In [ ]:
for i in range(len(padding_maxword[:40])):
  print(padding_maxword[i])
  print(X[i])
  print()

[10, 66, 6949, 8033, 183, 68, 6664, 42, 5, 1266, 6, 2, 1564, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
How did Quebec nationalists see their province as a nation in the ####s ? 

[58, 16, 26, 36, 3871, 522, 12, 79, 46, 16, 3640, 44, 4, 3067, 11, 29, 1899, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Do you have an adopted dog ,  how would you encourage people to adopt and not shop ? 

[17, 32, 2023, 390, 89, 1, 90, 2023, 390, 515, 5593, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Why does velocity affect time ?  Does velocity affect space geometry ? 

[10, 66, 14194, 10404, 56978, 139, 2, 42944, 30063, 1, 0, 0, 0, 0,

Train-test-split

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_validation,Y_train,Y_validation = train_test_split(np.array(padding_maxword),clean_train.target.values,test_size=0.15,shuffle=True)

In [ ]:
from tensorflow.keras import layers
import keras
import tensorflow as tf

from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

# This is called to clear the original model session in order to use TensorBoard
from tensorflow.keras import backend as K
K.clear_session()

# Path to save model parameters

int_sequences_input = keras.Input(shape=(70,), dtype=float)
embedded_sequences = layers.Embedding(100000,300,trainable=True)(int_sequences_input)
x = layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(128,return_sequences=True))(embedded_sequences)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.1)(x)
preds = layers.Dense(1, activation="sigmoid")(x)
model2 = keras.Model(int_sequences_input, preds)
model2.compile(optimizer=keras.optimizers.Adam(),loss='binary_crossentropy',metrics=['acc'])
model2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 70)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 70, 300)           30000000  
_________________________________________________________________
bidirectional (Bidirectional (None, 70, 256)           440320    
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65    

In [ ]:
model2.fit(X_train,Y_train, validation_data=(X_validation,Y_validation),
          epochs=3, batch_size=512,verbose=1)

Epoch 1/3
 240/2169 [==>...........................] - ETA: 11:00 - loss: 0.2385 - acc: 0.9322

KeyboardInterrupt: ignored

In [ ]:
y_predict1 = model2.predict(X_validation)
y_predict = pd.DataFrame(y_predict1,columns=["Y_prediction"])
y_predict

In [ ]:
y_predict["Y_prediction"] = np.where(y_predict.Y_prediction.values>=0.5,1,0)

In [ ]:
from sklearn.metrics import f1_score
f1score = f1_score(Y_validation,y_predict.Y_prediction.values)
print("F1_score of the model is ",f1score)

Load glove word embedding

Create new word index for Glove embedding

In [ ]:
crate_dict_for_glove = sort_dictionary(X,lower=False)
max_word_per_sentence = 70
number_of_word = 100000
dict_glove = {"OOV":0}
count = 1
for i in crate_dict_for_glove:
  count +=1
  dict_glove[i] =len(dict_glove)
  if count ==100000:
    break

In [ ]:
glove_padding = []
for sentence in X:
  list2 =[]
  for word in sentence.split():
    try:
      list2.append(dict_glove[word])
    except:
      list2.append(dict_glove["OOV"])
  glove_padding.append(list2)

In [ ]:
for j in glove_padding[:20]:
  print(j)

[10, 66, 6949, 8033, 183, 68, 6664, 42, 5, 1266, 6, 2, 1564, 1]
[58, 16, 26, 36, 3871, 522, 12, 79, 46, 16, 3640, 44, 4, 3067, 11, 29, 1899, 1]
[17, 32, 2023, 390, 89, 1, 90, 2023, 390, 515, 5593, 1]
[10, 66, 14194, 10404, 56978, 139, 2, 42944, 30063, 1]
[48, 9, 1119, 65729, 0, 1227, 4, 5, 4045, 1588, 56, 120, 1485, 2, 10057, 1]
[21, 9305, 3899, 778, 20494, 12, 56979, 27, 0, 15, 2626, 1]
[17, 32, 125, 2568, 1697, 1633, 2445, 51, 4337, 12, 81, 32, 29, 13, 2, 159, 15, 1278, 917, 1]
[21, 18, 1997, 37, 9, 3002, 27, 5887, 23, 13288, 247, 1, 80044, 14, 3730, 19]
[21, 47, 231, 5, 212, 42, 6745, 20495, 12, 11, 37, 65, 12, 79, 7, 22, 151, 88, 6745, 26962, 1]
[21, 18, 120, 67, 27, 26, 16, 112, 135, 6, 80, 2304, 16045, 16, 1392, 3166, 4, 2, 44, 16, 713, 1723, 12, 914, 24036, 68, 1083, 57, 1100, 65, 16, 38, 4, 26, 220, 129, 34, 99, 11, 121, 7082, 53, 6720, 19, 10, 66, 164, 190, 1]
[3, 20, 16, 177, 54, 2779, 1]
[10, 115, 2, 14037, 31271, 3175, 1]
[3, 7, 2, 7111, 12, 692, 2627, 229, 2643, 15, 143, 1

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
padding_maxword_for_glove = []
for i in glove_padding:
  sentence  = i.copy()
  if len(i) > max_word_per_sentence:
    padding_maxword_for_glove.append(sentence[:max_word_per_sentence])
  else:
    for i in range(max_word_per_sentence-len(i)):
      sentence.append(0)
    padding_maxword_for_glove.append(sentence)
X_train,X_validation,Y_train,Y_validation = train_test_split(np.array(padding_maxword_for_glove),clean_train.target.values,test_size=0.15,shuffle=True)

In [ ]:
for i in padding_maxword_for_glove[:2]:
  print(i)

[10, 66, 6949, 8033, 183, 68, 6664, 42, 5, 1266, 6, 2, 1564, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[58, 16, 26, 36, 3871, 522, 12, 79, 46, 16, 3640, 44, 4, 3067, 11, 29, 1899, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
def load_glove_fast(word_index, max_words, embed_size=300):
    EMBEDDING_FILE = 'embedding/glove.840B.300d/glove.840B.300d.txt'
    emb_mean, emb_std = -0.005838499, 0.48782197

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_words, embed_size))
    with open(EMBEDDING_FILE, 'r', encoding="utf8") as f:
        for line in f:
            word, vec = line.split(' ', 1)
            if word not in word_index:
                continue
            i = word_index[word]
            if i >= max_words:
                continue
            embedding_vector = np.asarray(vec.split(' '), dtype='float32')[:300]
            if len(embedding_vector) == 300:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [ ]:
Embedding_matrix = load_glove_fast(dict_glove,100000)
Embedding_matrix

array([[-0.51004189,  0.48892707, -0.39317015, ..., -0.33855094,
        -0.08941182, -1.10924053],
       [-0.086864  ,  0.19160999,  0.10915   , ..., -0.01516   ,
         0.11108   ,  0.20649999],
       [ 0.27204001, -0.06203   , -0.1884    , ...,  0.13015001,
        -0.18317001,  0.1323    ],
       ...,
       [-0.26304642, -0.34613858, -1.18406253, ...,  1.57106991,
        -0.42640117, -0.28469739],
       [-0.057969  , -0.14964999,  0.082356  , ..., -0.09207   ,
         0.10385   ,  0.16963001],
       [-0.067399  , -0.1991    , -0.11715   , ...,  0.36135   ,
        -0.12372   ,  0.044732  ]])

In [ ]:
from tensorflow.keras import layers
import keras
import tensorflow as tf

from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

# This is called to clear the original model session in order to use TensorBoard
from tensorflow.keras import backend as K
K.clear_session()

# Path to save model parameters


int_sequences_input = keras.Input(shape=(70,), dtype=float)
embedded_sequences = layers.Embedding(100000,300,weights=[Embedding_matrix],trainable=True,)(int_sequences_input)
x = layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(64,return_sequences=True))(embedded_sequences)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dense(16, activation="relu")(x)
x = layers.Dropout(0.1)(x)
preds = layers.Dense(1, activation="sigmoid")(x)
new_model = keras.Model(int_sequences_input, preds)
new_model.compile(optimizer=keras.optimizers.Adam(),loss='binary_crossentropy',metrics=['acc'])
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 70)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 70, 300)           30000000  
_________________________________________________________________
bidirectional (Bidirectional (None, 70, 128)           187392    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                2064      
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17    

In [ ]:
new_model.fit(X_train,Y_train, validation_data=(X_validation,Y_validation),
          epochs=2, batch_size=512,verbose=1)

Epoch 1/2
2169/2169 [==============================] - 658s 301ms/step - loss: 0.1337 - acc: 0.9486 - val_loss: 0.1007 - val_acc: 0.9593
Epoch 2/2
2169/2169 [==============================] - 659s 304ms/step - loss: 0.0921 - acc: 0.9633 - val_loss: 0.1002 - val_acc: 0.9601


In [ ]:
y_predict_glove = new_model.predict(X_validation)
y_predict1 = pd.DataFrame(y_predict_glove,columns=["Y_prediction_Proba"])
y_predict1["Y_prediction_final"] = np.where(y_predict1.Y_prediction_Proba.values>=0.5,1,0)


In [ ]:
y_predict1.to_csv("Glove_Prediction.csv",index=False)

In [ ]:
f1score = f1_score(Y_validation,y_predict1.Y_prediction_final.values)
print("F1_score of the model is ",f1score)

F1_score of the model is  0.6685038368592869


In [ ]:
print("Find the best F1-score model")
for i in range(20,70,1):
  y_predict = pd.DataFrame(y_predict_glove,columns=["Y_prediction"])
  y_predict["Y_prediction"] = np.where(y_predict.Y_prediction.values>=(i/100),1,0)
  f1score = f1_score(Y_validation,y_predict.Y_prediction.values)

  print("F1 score of boundary = "+str(i)+"% is "+str(f1score))
  print()


Find the best F1-score model
F1 score of boundary = 20% is 0.6626349026028268

F1 score of boundary = 21% is 0.6644385929376815

F1 score of boundary = 22% is 0.6669851887243192

F1 score of boundary = 23% is 0.6686879823594267

F1 score of boundary = 24% is 0.6692409378696167

F1 score of boundary = 25% is 0.6712093006919322

F1 score of boundary = 26% is 0.6732974656661476

F1 score of boundary = 27% is 0.6745182012847966

F1 score of boundary = 28% is 0.6755171793488037

F1 score of boundary = 29% is 0.6757295631683887

F1 score of boundary = 30% is 0.6767296516940171

F1 score of boundary = 31% is 0.6777875975843276

F1 score of boundary = 32% is 0.6790953318230771

F1 score of boundary = 33% is 0.6795317700736752

F1 score of boundary = 34% is 0.679504088630968

F1 score of boundary = 35% is 0.6795173408211278

F1 score of boundary = 36% is 0.6796012679983195

F1 score of boundary = 37% is 0.680184686417853

F1 score of boundary = 38% is 0.6803116641469938

F1 score of boundary = 

In [ ]:
print("F1 score of boundary = 34% is 0.6755271465876758")

F1 score of boundary = 34% is 0.6755271465876758


In [ ]:
def load_fasttext_fast(word_index, max_words=100000, embed_size=300):
    EMBEDDING_FILE = 'embedding/wiki-news-300d-1M/wiki-news-300d-1M.vec'
    emb_mean, emb_std = -0.0033470048, 0.109855264

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_words, embed_size))
    with open(EMBEDDING_FILE, 'r', encoding="utf8") as f:       
        for line in f:
            if len(line) <= 100:
                continue
            word, vec = line.split(' ', 1)
            if word not in word_index:
                continue
            i = word_index[word]
            if i >= max_words:
                continue
            embedding_vector = np.asarray(vec.split(' '), dtype='float32')[:300]
            if len(embedding_vector) == 300:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [ ]:
#use the same dictionary for fasttext
embedding_matrix = load_fasttext_fast(dict_glove,100000)

K.clear_session()


int_sequences_input = keras.Input(shape=(70,), dtype=float)
embedded_sequences = layers.Embedding(100000,300,weights=[Embedding_matrix],trainable=True,)(int_sequences_input)
x = layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(64,return_sequences=True))(embedded_sequences)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dense(16, activation="relu")(x)
x = layers.Dropout(0.1)(x)
preds = layers.Dense(1, activation="sigmoid")(x)
new_model2 = keras.Model(int_sequences_input, preds)
new_model2.compile(optimizer=keras.optimizers.Adam(),loss='binary_crossentropy',metrics=['acc'])
new_model2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 70)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 70, 300)           30000000  
_________________________________________________________________
bidirectional (Bidirectional (None, 70, 128)           187392    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                2064      
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17    

In [ ]:
new_model2.fit(X_train,Y_train, validation_data=(X_validation,Y_validation),
          epochs=2, batch_size=512,verbose=2)

Epoch 1/2
2169/2169 - 692s - loss: 0.1159 - acc: 0.9541 - val_loss: 0.1014 - val_acc: 0.9588
Epoch 2/2
2169/2169 - 694s - loss: 0.0927 - acc: 0.9629 - val_loss: 0.0994 - val_acc: 0.9603


In [ ]:
y_predict_fasttext = new_model2.predict(X_validation)
y_predict = pd.DataFrame(y_predict_fasttext,columns=["Y_prediction"])
y_predict["Y_prediction_final"] = np.where(y_predict.Y_prediction.values>=0.5,1,0)


In [ ]:
y_predict_fasttext

array([[4.0429831e-02],
       [3.9071412e-04],
       [4.7073155e-04],
       ...,
       [1.6899848e-03],
       [6.4358443e-05],
       [1.7116500e-04]], dtype=float32)

In [ ]:
f1score = f1_score(Y_validation,y_predict.Y_prediction_final.values)
print("F1_score of the model is ",f1score)

F1_score of the model is  0.6532470430707432


In [ ]:
print("Find the best F1-score model")
for i in range(20,70,1):
  y_predict = pd.DataFrame(y_predict_fasttext,columns=["Y_prediction"])
  y_predict["Y_prediction"] = np.where(y_predict.Y_prediction.values>=(i/100),1,0)
  f1score = f1_score(Y_validation,y_predict.Y_prediction.values)

  print("F1 score of boundary = "+str(i)+"% is "+str(f1score))
  print()

Find the best F1-score model
F1 score of boundary = 20% is 0.6549534432270588

F1 score of boundary = 21% is 0.6581595255547411

F1 score of boundary = 22% is 0.661064425770308

F1 score of boundary = 23% is 0.6635922661747626

F1 score of boundary = 24% is 0.6654688687720267

F1 score of boundary = 25% is 0.6681807064742672

F1 score of boundary = 26% is 0.6694708564561167

F1 score of boundary = 27% is 0.6706920575940548

F1 score of boundary = 28% is 0.6723720795868992

F1 score of boundary = 29% is 0.6737878235508568

F1 score of boundary = 30% is 0.6747689021470925

F1 score of boundary = 31% is 0.6752483351315153

F1 score of boundary = 32% is 0.6752290821691452

F1 score of boundary = 33% is 0.6765319673686208

F1 score of boundary = 34% is 0.6765167464114833

F1 score of boundary = 35% is 0.6769040343480447

F1 score of boundary = 36% is 0.6781990706392284

F1 score of boundary = 37% is 0.6772069550132083

F1 score of boundary = 38% is 0.6760911118190507

F1 score of boundary =

In [ ]:
print("Best F1 score of Fasttext at boundary = 37% is ",0.6770760506486445)

Best F1 score of Fasttext at boundary = 37% is  0.6770760506486445


Paragram_dict

In [ ]:
def load_para_fast(word_index, max_words=100000, embed_size=300):
    EMBEDDING_FILE = 'embedding/paragram_300_sl999/paragram_300_sl999.txt'
    emb_mean, emb_std = -0.0053247833, 0.49346462

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_words, embed_size))
    with open(EMBEDDING_FILE, 'r', encoding="utf8", errors="ignore") as f:        
        for line in f:
            if len(line) <= 100:
                continue
            word, vec = line.split(' ', 1)
            if word not in word_index:
                continue
            i = word_index[word]
            if i >= max_words:
                continue
            embedding_vector = np.asarray(vec.split(' '), dtype='float32')[:300]
            if len(embedding_vector) == 300:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [ ]:
paragram_dict = sort_dictionary(X,lower=True)
paragram_dict

{'?': 1381192,
 'the': 665695,
 'what': 471275,
 'is': 471008,
 'a': 411421,
 'to': 407953,
 'in': 378109,
 'of': 333513,
 'i': 331007,
 'how': 290455,
 'do': 269402,
 'and': 257901,
 'are': 249024,
 ',': 235096,
 'for': 204464,
 'you': 202227,
 'can': 178103,
 'why': 163946,
 'it': 147606,
 '.': 134056,
 'my': 112645,
 'that': 107758,
 'have': 99800,
 'if': 98952,
 'with': 95920,
 'on': 94851,
 'or': 94597,
 'does': 92652,
 'be': 92089,
 'not': 86718,
 '"': 73272,
 '#': 70598,
 'from': 69848,
 'your': 67682,
 'an': 67354,
 '-': 67094,
 'which': 65876,
 'should': 65675,
 'would': 63185,
 'when': 62903,
 'get': 62739,
 'best': 62476,
 'as': 60533,
 '(': 56618,
 ')': 56594,
 'people': 55725,
 '##': 55397,
 'some': 54458,
 'there': 54408,
 'will': 53308,
 'who': 52259,
 'like': 49484,
 'at': 47937,
 'about': 45165,
 'did': 45103,
 'they': 44459,
 'by': 42706,
 'was': 42365,
 '/': 42137,
 'we': 40790,
 'any': 40157,
 'so': 39065,
 'good': 38439,
 'me': 36606,
 'their': 34969,
 'am': 33886,

In [ ]:
embedding_layer = load_para_fast(paragram_dict,100000)


int_sequences_input = keras.Input(shape=(70,), dtype=float)
embedded_sequences = layers.Embedding(100000,300,weights=[Embedding_matrix],trainable=True,)(int_sequences_input)
x = layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(64,return_sequences=True))(embedded_sequences)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dense(16, activation="relu")(x)
x = layers.Dropout(0.1)(x)
preds = layers.Dense(1, activation="sigmoid")(x)
new_model3 = keras.Model(int_sequences_input, preds)
new_model3.compile(optimizer=keras.optimizers.Adam(),loss='binary_crossentropy',metrics=['acc'])
new_model3.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 70)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 70, 300)           30000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 70, 128)           187392    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17  

In [ ]:
new_model3.fit(X_train,Y_train, validation_data=(X_validation,Y_validation),
          epochs=2, batch_size=512,verbose=2)

Epoch 1/2
2169/2169 - 665s - loss: 0.1150 - acc: 0.9555 - val_loss: 0.1026 - val_acc: 0.9594
Epoch 2/2
2169/2169 - 658s - loss: 0.0925 - acc: 0.9631 - val_loss: 0.1002 - val_acc: 0.9601


In [ ]:
y_predict_paragram = new_model3.predict(X_validation)
y_predict = pd.DataFrame(y_predict_paragram,columns=["Y_prediction"])
y_predict["Y_prediction_final"] = np.where(y_predict.Y_prediction.values>=0.5,1,0)

In [ ]:
y_predict

,Y_prediction,Y_prediction_final
0,0.062933,0
1,0.000192,0
2,0.000356,0
3,0.230677,0
4,0.000074,0
...,...,...
195914,0.001801,0
195915,0.005098,0
195916,0.000912,0
195917,0.000051,0


In [ ]:
f1score = f1_score(Y_validation,y_predict.Y_prediction_final.values)
print("F1_score of the model is ",f1score)

F1_score of the model is  0.6591930207197382


In [ ]:
print("Find the best F1-score model")
for i in range(20,70,1):
  y_predict = pd.DataFrame(y_predict_paragram,columns=["Y_prediction"])
  y_predict["Y_prediction"] = np.where(y_predict.Y_prediction.values>=(i/100),1,0)
  f1score = f1_score(Y_validation,y_predict.Y_prediction.values)

  print("F1 score of boundary = "+str(i)+"% is"+str(f1score))
  print()

Find the best F1-score model
F1 score of boundary = 20% is0.664916646780077

F1 score of boundary = 21% is0.6671032676134013

F1 score of boundary = 22% is0.6677355268202573

F1 score of boundary = 23% is0.6697923643670463

F1 score of boundary = 24% is0.6714402764418795

F1 score of boundary = 25% is0.6729476068898559

F1 score of boundary = 26% is0.6746716970180657

F1 score of boundary = 27% is0.6760099531615925

F1 score of boundary = 28% is0.6772147230213175

F1 score of boundary = 29% is0.6780518659076533

F1 score of boundary = 30% is0.6790600052471797

F1 score of boundary = 31% is0.6796644751756972

F1 score of boundary = 32% is0.6797699310554983

F1 score of boundary = 33% is0.679745085995086

F1 score of boundary = 34% is0.679368029739777

F1 score of boundary = 35% is0.6789699570815451

F1 score of boundary = 36% is0.6792986870036953

F1 score of boundary = 37% is0.6781204227861131

F1 score of boundary = 38% is0.676268751994893

F1 score of boundary = 39% is0.6750583125552

In [ ]:
print("Best F1 score at boundary = 28% is",0.6730135210224115)

Best F1 score at boundary = 28% is 0.6730135210224115


Use three word_embedding_layer at the same_time & Basic Boosting algorithm


In [ ]:
coefficient = [0.33,0.33,0.34] #for glove fasttext and paragram

In [ ]:
basic_ensemble = 0.33*y_predict_glove+0.33*y_predict_fasttext+0.34*y_predict_paragram

In [ ]:
ensemble_df = pd.DataFrame(basic_ensemble,columns=["Y_prediction_Proba"])

In [ ]:
print("Find the best F1-score model")
for i in range(20,70,1):
  ensemble_df["Y_prediction"] = np.where(ensemble_df.Y_prediction_Proba.values >=(i/100),1,0)
  f1score = f1_score(Y_validation,ensemble_df.Y_prediction.values)
  print("F1 score of boundary = "+str(i)+"% is "+str(f1score))
  print()

Find the best F1-score model
F1 score of boundary = 20% is 0.6665555518517283

F1 score of boundary = 21% is 0.6689366866698131

F1 score of boundary = 22% is 0.6714865463970263

F1 score of boundary = 23% is 0.6731538992408558

F1 score of boundary = 24% is 0.675384454440841

F1 score of boundary = 25% is 0.6775115565122269

F1 score of boundary = 26% is 0.6785447029897017

F1 score of boundary = 27% is 0.6790416576732139

F1 score of boundary = 28% is 0.6809515162520429

F1 score of boundary = 29% is 0.6818548091714893

F1 score of boundary = 30% is 0.6822015090989793

F1 score of boundary = 31% is 0.6838863797543951

F1 score of boundary = 32% is 0.6850047036688617

F1 score of boundary = 33% is 0.6842624943035089

F1 score of boundary = 34% is 0.6849398974044866

F1 score of boundary = 35% is 0.6852731132985911

F1 score of boundary = 36% is 0.68545405005644

F1 score of boundary = 37% is 0.685420590081607

F1 score of boundary = 38% is 0.6857866213062225

F1 score of boundary = 39

In [ ]:
print("F1 score of boundary = 38% is 0.6857866213062225")

F1 score of boundary = 38% is 0.6857866213062225


In [ ]:
def find_best_score(dataframe):
  list1 = []
  for i in range(30,50,1):
    dataframe["Y_prediction"] = np.where(dataframe.Y_prediction_Proba.values >=(i/100),1,0)
    f1score = f1_score(Y_validation,dataframe.Y_prediction.values)
    list1.append([i,f1score])
  list_a = sorted(list1,key = lambda x:x[1],reverse=True)
  return list_a[0]

In [ ]:
find_best_score(ensemble_df)

[38, 0.6857866213062225]

In [ ]:
list_a = []
for i in np.arange(0.20,0.50,0.01):
  for j in np.arange(0.20,0.50,0.01):
    for k in np.arange(0.20,0.50,0.01):
      i = round(i,2)
      j = round(j,2)
      k = round(k,2)
      if i+j+k ==1:
        basic_ensemble2 = i*y_predict_glove + j*y_predict_fasttext + k*y_predict_paragram
        ensemble_df2 = pd.DataFrame(basic_ensemble2,columns=["Y_prediction_Proba"])
        best_f1score_for_coefficient = find_best_score(ensemble_df2)
        f1_best_score_coefficient = best_f1score_for_coefficient[1]
        best_boundary_for_coefficient = best_f1score_for_coefficient[0]
        coefficient = [i,j,k] #glove fasttext paragram
        list_a.append([coefficient,best_boundary_for_coefficient,f1_best_score_coefficient])



In [ ]:
sort_list_a = sorted(list_a,key=lambda x:x[2],reverse=True)

In [ ]:
best_parameter = sort_list_a[0][0]
best_parameter

[0.38, 0.21, 0.41]

In [ ]:
list_ab = sort_list_a[0]
list_ab

[[0.38, 0.21, 0.41], 35, 0.6864818960858489]

In [ ]:
len(list_a)

655

In [ ]:
parameter_tuning_Answer = best_parameter[0]*y_predict_glove + best_parameter[1]*y_predict_fasttext + best_parameter[2]*y_predict_paragram

In [ ]:
parameter_tuning_Answer

array([[6.8212405e-02],
       [2.9087716e-04],
       [3.8961406e-04],
       ...,
       [1.5532168e-03],
       [7.0507696e-05],
       [7.8445752e-05]], dtype=float32)

In [ ]:
ensemble_df_best_tuning = pd.DataFrame(parameter_tuning_Answer,columns=["Y_prediction_Proba"])
ensemble_df_best_tuning["Y_prediction"] = np.where(ensemble_df_best_tuning.Y_prediction_Proba>=0.34,1,0)

In [ ]:
ensemble_df_best_tuning

,Y_prediction_Proba,Y_prediction
0,0.068212,0
1,0.000291,0
2,0.000390,0
3,0.194979,0
4,0.000069,0
...,...,...
195914,0.001597,0
195915,0.005696,0
195916,0.001553,0
195917,0.000071,0


In [ ]:
print("Best Coefficient of Glove = ",list_ab[0][0])
print("Best Coefficient of Fasttext = ",list_ab[0][1])
print("Best Coefficient of Paragram = ",list_ab[0][2])
print("Boundary = ",list_ab[1])
print("Best F1 score = ",find_best_score(ensemble_df_best_tuning)[1])
print("Boundary found to check again",find_best_score(ensemble_df_best_tuning)[0])

Best Coefficient of Glove =  0.38
Best Coefficient of Fasttext =  0.21
Best Coefficient of Paragram =  0.41
Boundary =  35
Best F1 score =  0.6864818960858489
Boundary found to check again 35


In [ ]:
clean_test = test.copy()
clean_test["question_text"]= clean_test.question_text.apply(lambda x: clean_text(x))
clean_test["question_text"]= clean_test.question_text.apply(lambda x: clean_numbers(x))

In [ ]:
clean_test["question_text"] = clean_test["question_text"].apply(lambda x: check_missspell(x))

In [ ]:
Test_X = clean_test.question_text.values

In [ ]:
Test_X

array(['Why do so many women become so rude and arrogant when they get just a little bit of wealth and power ? ',
       'When should I apply for RV college of engineering and BMS college of engineering ?  Should I wait for the COMEDK result or am I supposed to apply before the result ? ',
       'What is it really like to be a nurse practitioner ? ', ...,
       'Where I can find best friendship quotes in Telugu ? ',
       'What are the causes of refraction of light ? ',
       'Climate change is a worrying topic .  How much time do we have left to find another planet ?  I mean ,  I do not think humans will survive on this earth for another #### years .  .  What do you think ? '],
      dtype=object)

In [ ]:
max_word_per_sentence = 70
test_padding = []
for sentence in Test_X:
  list2 =[]
  for word in sentence.split():
    try:
      list2.append(dict_use[word])
    except:
      list2.append(dict_use["OOV"])
  test_padding.append(list2)

In [ ]:
max_word_per_sentence = 70
test_padding_maxword = []
for i in test_padding:
  sentence  = i.copy()
  if len(i) > max_word_per_sentence:
    test_padding_maxword.append(sentence[:max_word_per_sentence])
  else:
    for i in range(max_word_per_sentence-len(i)):
      sentence.append(0)
    test_padding_maxword.append(sentence)

In [ ]:
for i in test_padding_maxword[:10]:
  print(i)

[17, 13, 65, 86, 145, 117, 65, 1811, 11, 4111, 51, 55, 38, 120, 5, 787, 1653, 8, 2226, 11, 329, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[128, 50, 9, 394, 15, 10606, 181, 8, 195, 11, 7581, 181, 8, 195, 1, 116, 9, 1568, 15, 2, 3980, 997, 27, 73, 9, 1611, 4, 394, 163, 2, 997, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[3, 7, 18, 152, 49, 4, 28, 5, 4570, 14149, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[93, 14, 3988, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[21, 499, 152, 377, 63, 44, 2340, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
len(test_padding_maxword)

375806

In [ ]:
y_predict_glove = new_model.predict(np.array(test_padding_maxword))
y_predict_fasttext = new_model2.predict(np.array(test_padding_maxword))
y_predict_paragram = new_model3.predict(np.array(test_padding_maxword))


In [ ]:
#Case1 Average case
Coefficient_Glove = 0.33
Coefficient_Fasttext =0.33
Coefficient_Paragram = 0.34
y_predict_proba1 = Coefficient_Glove*y_predict_glove + Coefficient_Fasttext*y_predict_fasttext + Coefficient_Paragram*y_predict_paragram
submission_1 = pd.DataFrame(y_predict_proba1,columns=["Y_prediction_Proba"])


In [ ]:
submission_to_answer = pd.read_csv("submission/sample_submission.csv")
submission_to_answer

,qid,prediction
0,0000163e3ea7c7a74cd7,0
1,00002bd4fb5d505b9161,0
2,00007756b4a147d2b0b3,0
3,000086e4b7e1c7146103,0
4,0000c4c3fbe8785a3090,0
...,...,...
375801,ffff7fa746bd6d6197a9,0
375802,ffffa1be31c43046ab6b,0
375803,ffffae173b6ca6bfa563,0
375804,ffffb1f7f1a008620287,0


In [ ]:
decision_boundary = 38 #for the first submission case
submission_1["Y_test_Predict"] = np.where(submission_1.Y_prediction_Proba >= decision_boundary/100,1,0)

In [ ]:
submission_1

,Y_prediction_Proba,Y_test_Predict
0,0.809963,1
1,0.000193,0
2,0.000149,0
3,0.000194,0
4,0.005191,0
...,...,...
375801,0.000195,0
375802,0.000246,0
375803,0.000766,0
375804,0.000031,0


In [ ]:
#Case 2 Optimization case
Coefficient_Glove = list_ab[0][0]
Coefficient_Fasttext = list_ab[0][1]
Coefficient_Paragram = list_ab[0][2]

In [ ]:
y_predict_proba2 = Coefficient_Glove*y_predict_glove + Coefficient_Fasttext*y_predict_fasttext + Coefficient_Paragram*y_predict_paragram
submission_2 = pd.DataFrame(y_predict_proba2,columns=["Y_prediction_Proba"])
decision_boundary2 = list_ab[1]
submission_2["Y_test_Predict"] = np.where(submission_2.Y_prediction_Proba >= decision_boundary2/100,1,0)

In [ ]:
submission_2

,Y_prediction_Proba,Y_test_Predict
0,0.809210,1
1,0.000195,0
2,0.000143,0
3,0.000190,0
4,0.005068,0
...,...,...
375801,0.000159,0
375802,0.000213,0
375803,0.000746,0
375804,0.000028,0


Answer in two submission

In [ ]:
submission_to_answer_case1 = submission_to_answer.copy()
submission_to_answer_case1["prediction"] = submission_1["Y_test_Predict"]

In [ ]:
submission_to_answer_case2 = submission_to_answer.copy()
submission_to_answer_case2["prediction"] = submission_2["Y_test_Predict"]

In [ ]:
submission_2.head(15)

,Y_prediction_Proba,Y_test_Predict
0,0.809210,1
1,0.000195,0
2,0.000143,0
3,0.000190,0
4,0.005068,0
5,0.009318,0
6,0.000238,0
7,0.000254,0
8,0.000274,0
9,0.000247,0


In [ ]:
submission_1.head(15)

,Y_prediction_Proba,Y_test_Predict
0,0.809963,1
1,0.000193,0
2,0.000149,0
3,0.000194,0
4,0.005191,0
5,0.008912,0
6,0.000252,0
7,0.000263,0
8,0.000311,0
9,0.000276,0


In [ ]:
submission_to_answer_case1.to_csv("Submission1.csv",index=False)
submission_to_answer_case1.head(15)

,qid,prediction
0,0000163e3ea7c7a74cd7,1
1,00002bd4fb5d505b9161,0
2,00007756b4a147d2b0b3,0
3,000086e4b7e1c7146103,0
4,0000c4c3fbe8785a3090,0
5,000101884c19f3515c1a,0
6,00010f62537781f44a47,0
7,00012afbd27452239059,0
8,00014894849d00ba98a9,0
9,000156468431f09b3cae,0


In [ ]:
submission_to_answer_case2.to_csv("Submission2.csv",index=False)
submission_to_answer_case2.head(15)

,qid,prediction
0,0000163e3ea7c7a74cd7,1
1,00002bd4fb5d505b9161,0
2,00007756b4a147d2b0b3,0
3,000086e4b7e1c7146103,0
4,0000c4c3fbe8785a3090,0
5,000101884c19f3515c1a,0
6,00010f62537781f44a47,0
7,00012afbd27452239059,0
8,00014894849d00ba98a9,0
9,000156468431f09b3cae,0


In [ ]:
submission_to_answer_case2.to_csv("Submission_Optimizing.csv",index=False)
file.downlaod

In [ ]:
submission_to_answer_case1

,qid,prediction
0,0000163e3ea7c7a74cd7,1
1,00002bd4fb5d505b9161,0
2,00007756b4a147d2b0b3,0
3,000086e4b7e1c7146103,0
4,0000c4c3fbe8785a3090,0
...,...,...
375801,ffff7fa746bd6d6197a9,0
375802,ffffa1be31c43046ab6b,0
375803,ffffae173b6ca6bfa563,0
375804,ffffb1f7f1a008620287,0
